The source code for this entry can be found at https://github.com/rctcwyvrn/math441-learning-portfolio/blob/master/cryptography_and_lattices.ipynb

In 1976 Whitfield Diffie and Martin Hellman developed what would become a key concept in modern telecommunications, the public key cryptosystem. In a paper titled "New directions in cryptography" they developed the first public key exchange and theorized what would eventually become public key cryptography but with one key part missing. Diffie and Hellman built their model for a public key cryptosystem with a theoretical "one way trapdoor function". A function that could be easily computed when given some secret information, but computationally infeasible to compute without. For many years after their seminal paper, cryptographers worked to develop this trapdoor function in reality.

In the theory of computational complexity, one of the most well studied areas of problems are NP problems, problems categorized by the ability to verify a correct answer in polynomial time. Many well known NP problems also lack fast polynomial time algorithms to solve, leading to many cryptographers attempting to build the aforementioned trapdoor function from NP problems. In theory this would work, NP problems are easily verifiable and yet lack efficient algorithms to solve from scratch.

In 1978 Ralph Merkle and Martin Hellman developed one of the earliest attempts at a public key cryptosystem, one that used a variation of the knapsack problem as its one way trapdoor function. 

## Background: Public key cryptography

Public key cryptography solves the problem of secure communication without a previously shared key. For example, if a user wants to connect to a server for the first time, how can they ensure that their connection is secure? Once they establish an initial secure connection they can share a key and proceed, but how should this initial encryption be made? The model developed by Diffie and Hellman uses the idea of a public and a private key pair. 

The theorized keys would have the following properties
1. The public key can be used to encrypt messages
2. A message encrypted by a public key can _only_ be decrypted by its corresponding private key

So the server that you'd like to connect to can generate this pair of keys and provide the public key out to the world. To send a secure message you would take that public key, encrypt your message, and send it to the server knowing that only the server can decrypt it, since only it has the private key.

The question of how to develop this encryption and key generation was the one proposed by Diffie and Hellman with their one way trapdoor, and the one that Merkle and Hellman attempted to answer.

## The Merkle-Hellman knapsack cryptosystem

As we talked about in class, the knapsack problem is where given a number of items with value and weights and a backpack of limited carrying capacity, decide how many of each item should be taken to maximize value. The subset sum problem is a special case of this problem, where given a multiset of integers and a target sum, decide whether or not there exists a subset of the given integers that sums to the target sum. Since both the knapsack problem and special cases of the subset sum problem are known to be NP-complete it was theorized that it would be a good candidate for a trapdoor function. 

The big picture idea of the Merkle-Hellman system is this 
1. Create a special set of weights $B$ and release this as the public key
2. To encrypt a message, use the weights and the message to create a sum $c$
3. To decrypt the message, one would have to solve the subset sum problem to find which values of $B$ sum to $c$, which was thought to be computationally infeasible
4. However, due to how the special set of weights $B$ is generated, the subset sum problem is easy to solve for the creator of the keys

So by making each message into an instance of the general subset sum problem, it appears that since there is no efficient algorithm for subset sum, there is no way to decrypt the message. This raises the question though, how does the server decrypt the message then? 

The answer lies in the generation of the keys.

Firstly, it is known that there is an efficient solution for the subset sum problem if the weights are _superincreasing_. A sequence $W$ is superincreasing if for each item in the sequence, it is greater than or equal to the sum of all prior elements, $\forall_i \sum_{j\lt i} w_j \leq w_i$. The reason why is pretty clear when we look at a greedy algorithm for this case. Consider a sequence of weights and the desired sum $c$

Consider the largest element in $W$ that is less than $c$, $w_k$

```
w_0 w_1 w_2 ... w_k   w_k+1
                    ^
                    |
                    c
```

$w_k$ must be greater than the sum of all elements before it, so the sum of all elements before $w_k$ are less than $c$. Therefore since all values are positive integers, $w_k$ must be included in a subset that sums to $c$ if it exists. So a greedy algorithm can take $w_k$, and then recursively solve for $c' = c - w_k$, giving us a polynomial time algorithm for finding the desired subset.

So from this you might be able to guess the idea behind the Merkle-Hellman cryptosystem
1. Create a set of superincreasing weights $W$
2. From this set, create another set of weights $B$ based on $W$ such that 
    - $B$ is not superincreasing
    - Given an instance of the subset sum problem on $B$, one can convert it to a subset sum problem on $W$
3. So given a message encrypted by the public key $B$, all the holder of the private key has to do is convert it to a problem on the superincreasing weights $W$ and solve it using the aforementioned greedy algorithm

That leaves us with one question, how do we generate this set of weights $B$?
- Since $B$ is released publically and are based on the private key $W$, this generation must be one-way
- We have to be able to reduce a subset sum problem on $B$ into a subset sum problem on $W$

The exact method to generate $B$ given the superincreasing private key set $W$ is
1. Choose a random integer $q \gt \sum_i w_i$
2. Choose a random integer $r$ such that $q$ and $r$ are co-prime
3. Compute $b_i = rw_i mod q$ for all $i$, the public key sequence is then $B = (b_1, b_2 ... b_n)$
4. Release $B$ as the public key weights

Does this satisfy the first property we need? That it's impossible to recover $w_i$ from $b_i$? We see that the answer is yes, since both the modulo $q$ and coefficient $r$ are unknown. 

Does this set give us an easy way to convert a subset sum problem on $B$ to one on $W$? Yes! An instance of the subset sum problem is just the target sum $T$ and the set.

First, define encryption of a sequence of bits $M = (m_1, m_2 \ldots m_n)$ as $c = \sum_i^n b_i m_i$.

To convert this to a sum over the weights $W$ we can compute the following
1. Compute $r' = r^{-1} (mod q)$ using the Extended Euclidean algorithm. Since we chose $r$ to be coprime to $q$, this inverse must exist
2. Compute $c' = c r' (mod q)$, we see that 
$\begin{align}
c' &= r' c (mod q) \\
   &= r^{-1} \sum_i^n m_i b_i \\
   &= r^{-1} \sum_i^n m_i r w_i \\
   &= \sum_i^n m_i w_i
\end{align}$

So we have our subset sum problem over $W$ as desired, and we can solve this efficiently using the aforementioned greedy algorithm. The chosen integers correspond with the bits of the message $m_i = 1$, so by setting the rest of the bits to be $0$ we can recover the entire message.

In [1]:
import random
import math
import numpy as np
from scipy.optimize import linprog
from Crypto.Util import number

In [2]:
# Decide on a private key, a superincreasing set of weights

def superincreasing_sequence(n):
    sum = 0
    seq = []
    for _ in range(n):
        next = random.randint(sum, sum+10)
        seq.append(next)
        sum += next
    return seq

In [3]:
# Generate the public key from those weights
def make_pubkey_from(W):
    total = sum(W)
    q = random.randint(total, total + 10)
    r = q
    while math.gcd(r,q) != 1:
        r = random.randint(10, q)
    
    pubkey = [r*w % q for w in W]
    privkey = (W, q, r)
    return pubkey, privkey

In [4]:
# Encrypt a message using the public key
# Assume message is a sequence of bytes
def encrypt(pubkey, message):
    hex_msg = message.hex()
    bits = bin(int(hex_msg, base=16)).lstrip('0b') # from stackoverflow
    padded = "0" * (8*len(message) - len(bits)) + bits # pad to the proper length
    sum = 0
    for bit,weight in zip(padded, pubkey):
        sum += int(bit) * weight
    return sum

In [5]:
# Decrypt the mesage using the private key

def greedy_superincreasing_weights_solver(W, c):
    c_current = c
    choices = []
    while True:
        if c_current == 0:
            return choices
        
        last = None
        for weight in W:
            if weight > c_current:
                # take the last weight, and solve the subproblem
                c_current -= last
                choices.append(last)
                break
            last = weight
        else:
            # c_current is greater than all elements in the array, take the last one
            c_current -= last
            choices.append(last)


def decrypt(privkey, c):
    W, q, r = privkey
    r_inv = number.inverse(r,q)
    c_prime = (r_inv * c) % q 
    subset = greedy_superincreasing_weights_solver(W, c_prime)
    message = ""
    for weight in W:
        if weight in subset:
            message += "1"
        else:
            message += "0"
    return number.long_to_bytes(int(message,2))


In [6]:
# attempt to solve the subset sum problem directly using linprog
def linprog_solve(W, sum):
    n = len(W)
    c_obj = np.zeros(n) # we just want a feasible solution
    A = np.array(W).reshape(1,n)
    b = np.array([sum])
    bounds = [(0,1) for _ in range(n)]
    integrality = np.ones(n)
    solution = linprog(c_obj, A_eq=A, b_eq=b, bounds=bounds, integrality=integrality)
    message_bits = "".join(["1" if b else "0" for b in solution.x])
    return number.long_to_bytes(int(message_bits,2))

In [7]:
# make a 16 weight superincreasing sequence
W = superincreasing_sequence(16)
W

# make the keypair out of it
pubkey, privkey = make_pubkey_from(W)
pubkey, privkey

# encrypt a 16 bit message
message = b"hi"
ciphertext = encrypt(pubkey, message)
ciphertext

# decrypt the message
recovered_message = decrypt(privkey, ciphertext)
recovered_message

b'hi'

In [8]:
# attempt to solve the longer message with linprog
linprog_solve(pubkey, ciphertext)

b'hi'

In [9]:
# try a longer message

message = b"MATH 441 is a project-based course which emphasizes mathematical research, communication, collaboration, computation and reflection."

W = superincreasing_sequence(8*len(message))
pubkey, privkey = make_pubkey_from(W)
ciphertext = encrypt(pubkey, message)
print("ciphertext = ", ciphertext)
recovered_message = decrypt(privkey, ciphertext)
print("decrypted message = ", recovered_message)

ciphertext =  536364990818258932943800609298354787715168485943509264980863553091867727082344099459967930841316809559412937252341166146607207289016042932811930253301092484554712921097687085585782642533819498260808656353443034214226752407487875922021363657631872717180143035219789177321849490070838974311046929258163148056471479301583555
decrypted message =  b'MATH 441 is a project-based course which emphasizes mathematical research, communication, collaboration, computation and reflection.'


In [10]:
# attempt to solve the longer message with linprog, we see that it fails!
linprog_solve(pubkey, ciphertext)

OverflowError: int too large to convert to float

# The fatal flaw: Lattices and lattice reductions

So why don't we use this cryptosystem today? Ignoring the issue of the massive key size, it turns out that there is a way of solving the subset sum problem $c$ without knowing anything besides the public key in polynomial time, rendering the entire system useless.

How does it work?

- Lattice definition
- the knapsack can be changed to a dot product, t backpack, u message, C value, t dot u = c is what we want to solve, u = {0,1}^n
- impossible to solve directly, but we can convert it into a matrix problem
    - let V = [u1 ... un, 1]
    - let W = [u1 ... un, 0]
    - let M = [ identity_n  0
                b_1 ... b_n -c]
    - we want to find W
- All elements of V are either zero or one, so W is a lattice in the basis formed by the columns of M
- note that W appears to be a short vector, since all values are either one or zero
- hypothesize that W = a basis vector of M
- compute the LLL reduced basis of M
- check if any basis vectors are a valid solution (all zeroes and ones, then one zero) -> if so then we have our solution

In [11]:
# lattice testing
W = superincreasing_sequence(4)
print(W)
pubkey, privkey = make_pubkey_from(W)
print(pubkey, privkey)
message = "0111"
ciphertext = sum([pubkey[i] if b == "1" else 0 for i,b in enumerate(message)])
print(ciphertext)

[7, 17, 30, 63]
[43, 122, 114, 18] ([7, 17, 30, 63], 123, 94)
254


In [12]:
# LLL solver, doesn't work :c
import fpylll

def lattice_solve(B, c):
    n = len(B)
    M_top = np.hstack([np.eye(n), np.zeros((n,1))])
    M_bottom = np.hstack([B, [-c]])
    M = np.vstack([M_top, M_bottom])
    M = fpylll.IntegerMatrix.from_matrix(M.astype(int).tolist())
    print(M)
    print("doing LLL")
    reduced = fpylll.LLL.reduction(M)
    print(reduced)

recovered_message = lattice_solve(pubkey, ciphertext)

[  1   0   0  0    0 ]
[  0   1   0  0    0 ]
[  0   0   1  0    0 ]
[  0   0   0  1    0 ]
[ 43 122 114 18 -254 ]
doing LLL
[ 1 0 0 0    0 ]
[ 0 1 0 0    0 ]
[ 0 0 1 0    0 ]
[ 0 0 0 1    0 ]
[ 0 0 0 0 -254 ]
